<a href="https://colab.research.google.com/github/Alexandremsn/demand/blob/main/cff_preprocessing%20P3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
traindf = pd.read_csv('train_selected_final.csv')
testdf = pd.read_csv('test_oil.csv')
holidaysdf = pd.read_csv('holidays_ok.csv')
itemsdf = pd.read_csv('items.csv')
sampledf = pd.read_csv('sample_submission.csv')
transactionsdf = pd.read_csv('transactions.csv')
oildf = pd.read_csv('oil_fixed.csv') 

finalizando a etapa de pré processamento dos dados, agora vamos contruir o dataset final que analizaremos e depois construiremos um modelo, primeiro importamos os arquivos

In [ ]:
sampledf.head()

,id,unit_sales
0,125497040,0
1,125497041,0
2,125497042,0
3,125497043,0
4,125497044,0


In [ ]:
transactionsdf.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [ ]:
traindf.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,82,2013-01-01,25,258396,5,False
1,84,2013-01-01,25,261052,1,False
2,92,2013-01-01,25,265559,2,False
3,98,2013-01-01,25,273528,2,False
4,116,2013-01-01,25,314384,8,False


In [ ]:
traindf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1634358 entries, 0 to 1634357
Data columns (total 6 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   id           1634358 non-null  int64 
 1   date         1634358 non-null  object
 2   store_nbr    1634358 non-null  int64 
 3   item_nbr     1634358 non-null  int64 
 4   unit_sales   1634358 non-null  int64 
 5   onpromotion  1634358 non-null  bool  
dtypes: bool(1), int64(4), object(1)
memory usage: 63.9+ MB


In [ ]:
traindf.date = pd.to_datetime(traindf.date, format='%Y-%m-%d')
transactionsdf.date = pd.to_datetime(transactionsdf.date, format='%Y-%m-%d')


In [ ]:

oildf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1218 non-null   object 
 1   dcoilwtico  1218 non-null   float64
dtypes: float64(1), object(1)
memory usage: 19.2+ KB


tranformamos o tipo da coluna date de todos os dataframes,

In [ ]:
oildf.date = pd.to_datetime(oildf.date, format='%Y-%m-%d')

In [ ]:
testdf.date = pd.to_datetime(testdf.date, format='%Y-%m-%d')

In [ ]:
holidaysdf.date = pd.to_datetime(holidaysdf.date, format='%Y-%m-%d')

vamos adicionar a coluna preço do petróleo nos datasets de treino e transactions

In [ ]:
traindf = traindf.merge(oildf,how='left', on='date')

In [ ]:
transactionsdf = transactionsdf.merge(oildf,how='left', on='date')

agora vamos criar uma variável de dia da semana

In [ ]:
traindf['day_name'] = traindf['date'].apply(lambda x: x.day_name())
testdf['day_name']  = testdf['date'].apply(lambda x: x.day_name())
transactionsdf['day_name']  = transactionsdf['date'].apply(lambda x: x.day_name())

In [ ]:
traindf.day_name.value_counts()

Saturday     236377
Monday       234620
Tuesday      234164
Sunday       233978
Wednesday    231974
Friday       231913
Thursday     231332
Name: day_name, dtype: int64

In [ ]:
traindf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1634358 entries, 0 to 1634357
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   id           1634358 non-null  int64         
 1   date         1634358 non-null  datetime64[ns]
 2   store_nbr    1634358 non-null  int64         
 3   item_nbr     1634358 non-null  int64         
 4   unit_sales   1634358 non-null  int64         
 5   onpromotion  1634358 non-null  bool          
 6   dcoilwtico   1164003 non-null  float64       
 7   day_name     1634358 non-null  object        
dtypes: bool(1), datetime64[ns](1), float64(1), int64(4), object(1)
memory usage: 101.3+ MB


In [ ]:
traindf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1634358 entries, 0 to 1634357
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   id           1634358 non-null  int64         
 1   date         1634358 non-null  datetime64[ns]
 2   store_nbr    1634358 non-null  int64         
 3   item_nbr     1634358 non-null  int64         
 4   unit_sales   1634358 non-null  int64         
 5   onpromotion  1634358 non-null  bool          
 6   dcoilwtico   1164003 non-null  float64       
 7   day_name     1634358 non-null  object        
dtypes: bool(1), datetime64[ns](1), float64(1), int64(4), object(1)
memory usage: 101.3+ MB


verificamos a presença de valores nulos na coluna com preço do petróleo, isso se deve ao fato de os valores de mercado serem cotados em dias úteis, assim vamos criar uma função para repetir o valor de sexta-feira para o final de semana.

In [ ]:
weekend_list = ['Saturday', 'Sunday']
for i in range(traindf.shape[0]):
  if traindf.day_name.iloc[i] == 'Friday':
    friday_price = traindf.dcoilwtico.iloc[i]
  elif traindf.day_name.iloc[i] in weekend_list:
    traindf.dcoilwtico.iloc[i] = friday_price



/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
for i in range(transactionsdf.shape[0]):
  if transactionsdf.day_name.iloc[i] == 'Friday':
    friday_price = transactionsdf.dcoilwtico.iloc[i]
  elif transactionsdf.day_name.iloc[i] in weekend_list:
    transactionsdf.dcoilwtico.iloc[i] = friday_price

In [ ]:
traindf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1634358 entries, 0 to 1634357
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   id           1634358 non-null  int64         
 1   date         1634358 non-null  datetime64[ns]
 2   store_nbr    1634358 non-null  int64         
 3   item_nbr     1634358 non-null  int64         
 4   unit_sales   1634358 non-null  int64         
 5   onpromotion  1634358 non-null  bool          
 6   dcoilwtico   1634358 non-null  float64       
 7   day_name     1634358 non-null  object        
dtypes: bool(1), datetime64[ns](1), float64(1), int64(4), object(1)
memory usage: 101.3+ MB


In [ ]:
holidaysdf.head()

,date,type,locale,locale_name,description,stores_affected,classification,major_holiday
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,"[52, 53]",is_holiday,NaN
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,"[12, 13]",is_holiday,NaN
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,"[37, 39, 42]",is_holiday,NaN
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,[36],is_holiday,NaN
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,[14],is_holiday,NaN


In [ ]:
testdf.head()

,id,date,store_nbr,item_nbr,onpromotion,oil_price,day_name
0,125497250,2017-08-16,1,258396,False,48.151379,Wednesday
1,125497256,2017-08-16,1,261052,False,48.151379,Wednesday
2,125497276,2017-08-16,1,265559,False,48.151379,Wednesday
3,125497294,2017-08-16,1,273528,False,48.151379,Wednesday
4,125497338,2017-08-16,1,314384,True,48.151379,Wednesday


In [ ]:
holidaysdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333 entries, 0 to 332
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   date             333 non-null    datetime64[ns]
 1   type             333 non-null    object        
 2   locale           333 non-null    object        
 3   locale_name      333 non-null    object        
 4   description      333 non-null    object        
 5   stores_affected  333 non-null    object        
 6   classification   333 non-null    object        
 7   major_holiday    44 non-null     object        
dtypes: datetime64[ns](1), object(7)
memory usage: 20.9+ KB


In [ ]:
holidaysdf.major_holiday.fillna('ordinary', inplace=True)

vamos preencher os valores null da major holiday com o string ordinary

In [ ]:
testdf.shape[0]

17280

In [ ]:
testdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17280 entries, 0 to 17279
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id           17280 non-null  int64         
 1   date         17280 non-null  datetime64[ns]
 2   store_nbr    17280 non-null  int64         
 3   item_nbr     17280 non-null  int64         
 4   onpromotion  17280 non-null  bool          
 5   oil_price    17280 non-null  float64       
 6   day_name     17280 non-null  object        
dtypes: bool(1), datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 827.0+ KB


In [ ]:
holidaysdf.iloc[315]

date                                             2017-10-09 00:00:00
type                                                         Holiday
locale                                                      National
locale_name                                                  Ecuador
description                               Independencia de Guayaquil
stores_affected    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...
classification                                            is_holiday
major_holiday                                               ordinary
Name: 315, dtype: object

In [ ]:
traindf.date.iloc[83920]

Timestamp('2013-04-04 00:00:00')

In [ ]:
indexof = traindf.date[traindf.date == holidaysdf['date'].iloc[50]].index.to_list()
indexof

[116595,
 116596,
 116597,
 116598,
 116599,
 116600,
 116601,
 116602,
 116603,
 116604,
 116605,
 116606,
 116607,
 116608,
 116609,
 116610,
 116611,
 116612,
 116613,
 116614,
 116615,
 116616,
 116617,
 116618,
 116619,
 116620,
 116621,
 116622,
 116623,
 116624,
 116625,
 116626,
 116627,
 116628,
 116629,
 116630,
 116631,
 116632,
 116633,
 116634,
 116635,
 116636,
 116637,
 116638,
 116639,
 116640,
 116641,
 116642,
 116643,
 116644,
 116645,
 116646,
 116647,
 116648,
 116649,
 116650,
 116651,
 116652,
 116653,
 116654,
 116655,
 116656,
 116657,
 116658,
 116659,
 116660,
 116661,
 116662,
 116663,
 116664,
 116665,
 116666,
 116667,
 116668,
 116669,
 116670,
 116671,
 116672,
 116673,
 116674,
 116675,
 116676,
 116677,
 116678,
 116679,
 116680,
 116681,
 116682,
 116683,
 116684,
 116685,
 116686,
 116687,
 116688,
 116689,
 116690,
 116691,
 116692,
 116693,
 116694,
 116695,
 116696,
 116697,
 116698,
 116699,
 116700,
 116701,
 116702,
 116703,
 116704,
 116705,
 

In [ ]:
testdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17280 entries, 0 to 17279
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id           17280 non-null  int64         
 1   date         17280 non-null  datetime64[ns]
 2   store_nbr    17280 non-null  int64         
 3   item_nbr     17280 non-null  int64         
 4   onpromotion  17280 non-null  bool          
 5   oil_price    17280 non-null  float64       
 6   day_name     17280 non-null  object        
dtypes: bool(1), datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 827.0+ KB


In [ ]:
holidaysdf.tail(50)

,date,type,locale,locale_name,description,stores_affected,classification,major_holiday
283,2016-12-25,Holiday,National,Ecuador,Navidad,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",is_holiday,ordinary
284,2016-12-26,Additional,National,Ecuador,Navidad+1,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",is_additional,ordinary
285,2016-12-31,Additional,National,Ecuador,Primer dia del ano-1,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",is_holiday_eve,None
286,2017-01-02,Transfer,National,Ecuador,Traslado Primer dia del ano,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",is_holiday,ordinary
287,2017-02-27,Holiday,National,Ecuador,Carnaval,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",is_holiday,ordinary
288,2017-02-28,Holiday,National,Ecuador,Carnaval,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",is_holiday,ordinary
289,2017-03-02,Holiday,Local,Manta,Fundacion de Manta,"[52, 53]",is_holiday,ordinary
290,2017-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,"[12, 13]",is_holiday,ordinary
291,2017-04-13,Transfer,Local,Cuenca,Fundacion de Cuenca,"[37, 39, 42]",is_holiday,ordinary
292,2017-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,[36],is_holiday,ordinary


In [ ]:
testdf.tail(50)

,id,date,store_nbr,item_nbr,onpromotion,oil_price,day_name
17230,128856455,2017-08-31,52,564533,False,48.151379,Thursday
17231,128856487,2017-08-31,52,581078,False,48.151379,Thursday
17232,128856712,2017-08-31,52,759893,False,48.151379,Thursday
17233,128856843,2017-08-31,52,839363,False,48.151379,Thursday
17234,128856857,2017-08-31,52,841842,True,48.151379,Thursday
17235,128856873,2017-08-31,52,847863,False,48.151379,Thursday
17236,128856886,2017-08-31,52,850333,False,48.151379,Thursday
17237,128857177,2017-08-31,52,1036689,True,48.151379,Thursday
17238,128857205,2017-08-31,52,1047679,False,48.151379,Thursday
17239,128857304,2017-08-31,52,1084881,False,48.151379,Thursday


ciamos esta função para preencher com as informações do dataset de feriados as linhas nas demais tabelas.

In [ ]:
testdf['classification'] = 'ordinary'
testdf['major_holiday'] = 'ordinary'
for i in range(testdf.shape[0]):
  currentdate = testdf['date'].iloc[i]
  
  if holidaysdf.date[holidaysdf['date'] == currentdate].index.to_list():
    indexof = holidaysdf.date[holidaysdf['date'] == currentdate].index.to_list()[0]
    
    if str(testdf['store_nbr'].iloc[i]) in holidaysdf['stores_affected'].iloc[indexof] :
      testdf['classification'].iloc[i] = holidaysdf['classification'].iloc[indexof]
      testdf['major_holiday'].iloc[i] = holidaysdf['major_holiday'].iloc[indexof]
  

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
transactionsdf['classification'] = 'ordinary'
transactionsdf['major_holiday'] = 'ordinary'
for i in range(transactionsdf.shape[0]):
  currentdate = transactionsdf['date'].iloc[i]
  
  if holidaysdf.date[holidaysdf['date'] == currentdate].index.to_list():
    indexof = holidaysdf.date[holidaysdf['date'] == currentdate].index.to_list()[0]
    
    if str(transactionsdf['store_nbr'].iloc[i]) in holidaysdf['stores_affected'].iloc[indexof] :
      transactionsdf['classification'].iloc[i] = holidaysdf['classification'].iloc[indexof]
      transactionsdf['major_holiday'].iloc[i] = holidaysdf['major_holiday'].iloc[indexof]

In [ ]:
traindf['classification'] = 'ordinary'
traindf['major_holiday'] = 'ordinary'
for i in range(traindf.shape[0]):
  currentdate = traindf['date'].iloc[i]
  
  if holidaysdf.date[holidaysdf['date'] == currentdate].index.to_list():
    indexof = holidaysdf.date[holidaysdf['date'] == currentdate].index.to_list()[0]
    
    if str(traindf['store_nbr'].iloc[i]) in holidaysdf['stores_affected'].iloc[indexof] :
      traindf['classification'].iloc[i] = holidaysdf['classification'].iloc[indexof]
      traindf['major_holiday'].iloc[i] = holidaysdf['major_holiday'].iloc[indexof]



In [ ]:
traindf.classification.value_counts()

ordinary               1502950
is_holiday               56205
is_natural_disaster      27668
is_holiday_eve           26346
is_world_cup             11952
is_additional             3315
is_cyber_monday           2986
is_black_friday           2936
Name: classification, dtype: int64

In [ ]:
traindf.rename(columns={'dcoilwtico':'oil_price'}, inplace=True)
traindf.head(1)


,id,date,store_nbr,item_nbr,unit_sales,onpromotion,oil_price,day_name,classification,major_holiday
0,82,2013-01-01,25,258396,5,False,67.714366,Tuesday,is_holiday,ordinary


In [ ]:
testdf.head(1)

,id,date,store_nbr,item_nbr,onpromotion,oil_price,day_name,classification,major_holiday
0,125497250,2017-08-16,1,258396,False,48.151379,Wednesday,ordinary,ordinary


In [ ]:
transactionsdf.rename(columns={'dcoilwtico':'oil_price'}, inplace=True)
transactionsdf.head(1)

,date,store_nbr,transactions,oil_price,day_name,classification,major_holiday
0,2013-01-01,25,770,67.714366,Tuesday,is_holiday,ordinary


agora vamos cruzar as tabelas de itens com as demais tabelas

In [ ]:
itemsdf.head()

,item_nbr,family,class,perishable
0,96995,GROCERY I,1093,0
1,99197,GROCERY I,1067,0
2,103501,CLEANING,3008,0
3,103520,GROCERY I,1028,0
4,103665,BREAD/BAKERY,2712,1


In [ ]:
list_best_sales = [502331, 314384, 364606, 265559, 559870,1036689,
                   273528 ,564533 ,261052 , 414353 , 1084881 , 839363 , 850333 , 414750 ,
                  847863 , 581078 , 1047679 , 258396 , 759893 , 841842]

separamos apenas os 20 produtos mais vendidos conforme consultamos antes com uma consulta sql

In [ ]:
itemsdf = itemsdf[itemsdf['item_nbr'].isin(list_best_sales)]

In [ ]:
itemsdf.head(50)

,item_nbr,family,class,perishable
212,258396,GROCERY I,1010,0
218,261052,GROCERY I,1072,0
240,265559,GROCERY I,1072,0
259,273528,PERSONAL CARE,4114,0
303,314384,GROCERY I,1004,0
394,364606,GROCERY I,1014,0
470,414353,GROCERY I,1014,0
481,414750,GROCERY I,1072,0
575,502331,BREAD/BAKERY,2702,1
659,559870,GROCERY I,1072,0


In [ ]:
dictfamily = dict(zip(itemsdf.item_nbr, itemsdf['family']))
dictclass = dict(zip(itemsdf.item_nbr, itemsdf['class']))
dictperishable = dict(zip(itemsdf.item_nbr, itemsdf['perishable']))

In [ ]:
traindf['family'] = traindf['item_nbr'].apply(lambda x: dictfamily[x])
traindf['class'] = traindf['item_nbr'].apply(lambda x: dictclass[x])
traindf['perishable'] = traindf['item_nbr'].apply(lambda x: dictperishable[x])
testdf['family'] = testdf['item_nbr'].apply(lambda x: dictfamily[x])
testdf['class'] = testdf['item_nbr'].apply(lambda x: dictclass[x])
testdf['perishable'] = testdf['item_nbr'].apply(lambda x: dictperishable[x])

In [ ]:
testdf.onpromotion = testdf.onpromotion.astype(int)
traindf.onpromotion = traindf.onpromotion.astype(int)

adicionamos as informação de produto as tabelas que utilizaremos em nosso modelo

In [ ]:
testdf.head(1)

,id,date,store_nbr,item_nbr,onpromotion,oil_price,day_name,classification,major_holiday,family,class,perishable
0,125497250,2017-08-16,1,258396,0,48.151379,Wednesday,ordinary,ordinary,GROCERY I,1010,0


In [ ]:
traindf.head(1)

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,oil_price,day_name,classification,major_holiday,family,class,perishable
0,82,2013-01-01,25,258396,5,0,67.714366,Tuesday,is_holiday,ordinary,GROCERY I,1010,0


In [ ]:
from sklearn.preprocessing import OneHotEncoder

criaremos as variáveis dummies

In [ ]:
ohe = OneHotEncoder(handle_unknown = 'ignore', sparse=False)
selected = ['day_name' ,	'classification',	'major_holiday',	'family',	'class']
X_ohetrain = traindf[selected]
X_ohetest = testdf[selected]
X_ohetrain = ohe.fit_transform(X_ohetrain)
X_ohetrain = pd.DataFrame(X_ohetrain, columns=ohe.get_feature_names())
X_ohetest = ohe.transform(X_ohetest)
X_ohetest = pd.DataFrame(X_ohetest, columns=ohe.get_feature_names())
selected_from_transctions = ['day_name' ,	'classification',	'major_holiday']
X_ohetransactions = transactionsdf[selected_from_transctions]
X_ohetransactions = ohe.fit_transform(X_ohetransactions)
X_ohetransactions = pd.DataFrame(X_ohetransactions, columns=ohe.get_feature_names())
X_ohetest.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,x0_Friday,x0_Monday,x0_Saturday,x0_Sunday,x0_Thursday,x0_Tuesday,x0_Wednesday,x1_is_additional,x1_is_black_friday,x1_is_cyber_monday,...,x4_1040,x4_1058,x4_1072,x4_1084,x4_1120,x4_1124,x4_2702,x4_3032,x4_3038,x4_4114
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
testdf.day_name.value_counts()

Wednesday    3240
Thursday     3240
Friday       2160
Saturday     2160
Sunday       2160
Monday       2160
Tuesday      2160
Name: day_name, dtype: int64

In [ ]:
def autorename(df):
  new_list = []
  old_list = df.columns 
  for i in (df.columns):
    new_list.append(i[3:])

  dictrename = dict(zip(old_list, new_list))
  df.rename(columns=dictrename, inplace=True)

In [ ]:
autorename(X_ohetrain)
autorename(X_ohetest)
autorename(X_ohetransactions)

criamos ums função para renomear as variáveis

criamos ums função para renomear as variáveis

In [ ]:
X_ohetransactions.head()

,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,is_additional,is_black_friday,is_cyber_monday,is_holiday,is_holiday_eve,is_natural_disaster,is_world_cup,ordinary,Madre,Navidad,None,ordinary
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [ ]:
X_ohetest.head(1)

,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,is_additional,is_black_friday,is_cyber_monday,...,1040,1058,1072,1084,1120,1124,2702,3032,3038,4114
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
X_ohetest.drop('None', inplace=True, axis=1)
X_ohetrain.drop('None', inplace=True, axis=1)
X_ohetransactions.drop('None', inplace=True, axis=1)

dropamos uma das variáveis dummies que estava redundante

In [ ]:
traindf.shape

(1634358, 13)

In [ ]:
X_ohetrain.shape

(1634358, 37)

In [ ]:
traindf =pd.concat([traindf,X_ohetrain], axis=1)
traindf.drop(selected, inplace=True, axis=1)

testdf =pd.concat([testdf,X_ohetest],axis=1)
testdf.drop(selected, inplace=True,axis=1)

transactionsdf =pd.concat([transactionsdf,X_ohetransactions],axis=1)
transactionsdf.drop(selected_from_transctions, inplace=True,axis=1)

juntamos o dataset com as variáveis dummies ao dataset original e dropamos as variáveis de origem

In [ ]:
traindf.head(1)

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,oil_price,perishable,Friday,Monday,...,1040,1058,1072,1084,1120,1124,2702,3032,3038,4114
0,82,2013-01-01,25,258396,5,0,67.714366,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
traindf.to_csv('train_processed.csv',index=False)
testdf.to_csv('test_processed.csv',index=False)
transactionsdf.to_csv('transactions_processed.csv',index=False)

exportamos para darmos continuidade em outra máquina